<a href="https://colab.research.google.com/github/marinebon/HackingLimno2025/blob/main/03_sdm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Modify this section as needed.
# ==============================================================================
# === If using google colab and google drive:
# TODO: is there a way to do this without granting excessive permissions?
from google.colab import drive
drive.mount('/content/drive')

# NOTE: Before running you must create a folder in your google drive
#       matching the directory name here `GSoC_SDM_Project`.
PROJECT_DIR = '/content/drive/MyDrive/GSoC_SDM_Project'

# === if using local machine
# PROJECT_DIR = './'
#import os
#if not os.path.exists(PROJECT_DIR):
#    os.makedirs(PROJECT_DIR)
# ==============================================================================
# ==============================================================================
# === spatial coverage
# ==============================================================================
# # === FL
LATMIN = 24.0
LATMAX = 30.7
LONMIN = -87.9
LONMAX = -79.5
# === FL Keys
# LATMIN = 24.11637699635014
# LATMAX = 26.11949526731449
# LONMIN = -82.51572158798965
# LONMAX = -79.61106009492724
# ==============================================================================

# ==============================================================================
# === install functions from github repo
# ==============================================================================
# Clone the GitHub repo
!git clone https://github.com/marinebon/HackingLimno2025.git

# import files from the repo
import sys
sys.path.append('/content/HackingLimno2025/py')

# imports from these files
# SDM Class for Marine species
from MarineSpeciesDistributionModel import MarineSpeciesDistributionModel
# ==============================================================================

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'HackingLimno2025' already exists and is not an empty directory.


In [2]:
# install dependencies TODO: are all these needed?
!pip install  rasterio pandas geopandas matplotlib  requests xarray netCDF4

Import libraries & setup


In [3]:
# TODO: are all these imports needed?


# Standard library imports
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Union
import warnings

# Third-party library imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import seaborn as sns
import xarray as xr
from scipy.spatial.distance import cdist
%matplotlib inline


In [4]:
df = pd.read_parquet(f'{PROJECT_DIR}/occurrences_and_environment.parquet')
environmental_cols = ['t']#, 'temperature_sd']  # TODO: can we get these dynamically from the df?

# Initialize  SDM
sdm = MarineSpeciesDistributionModel(nu=0.1, kernel='rbf', gamma=0.5)
sdm.species_name = 'Species'

# Prepare the data
data = sdm.prepare_data(df, environmental_cols)

# Fit the model
sdm.fit(data['X_train'])

# Evaluate the model
auc_score, fpr, tpr = sdm.evaluate_model(data['X_test'])
print(f"Model AUC Score: {auc_score:.3f}")

# Create prediction grid
grid_resolution = 0.01
bounds = (LONMIN, LATMIN, LONMAX, LATMAX)

# Create a simple prediction grid
lons = np.arange(bounds[0], bounds[2], grid_resolution)
lats = np.arange(bounds[1], bounds[3], grid_resolution)
lon_grid, lat_grid = np.meshgrid(lons, lats)

grid_coords = np.column_stack([lon_grid.ravel(), lat_grid.ravel()])

# Simulate environmental data for grid
# TODO: this needs to be loaded from the WOA files
#grid_salinity = np.random.uniform(32, 36, len(grid_coords))
grid_temperature = np.random.uniform(10, 25, len(grid_coords))

grid_env = np.column_stack([grid_temperature])  #[grid_salinity, grid_temperature])

# Make predictions
predictions = sdm.predict(grid_env)

# Create grid DataFrame
grid_df = pd.DataFrame({
    'longitude': grid_coords[:, 0],
    'latitude': grid_coords[:, 1],
#    'salinity': grid_salinity,
    'temperature': grid_temperature,
    'suitability': predictions
})

# Plot the results
fig = sdm.plot_distribution_map(grid_df, occurrence_data=data)



df_results = grid_df

print("\n Creating visualizations...")

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc_score:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
scatter = plt.scatter(
    df_results['longitude'],
    df_results['latitude'],
    c=df_results['suitability'],
    cmap='RdYlBu_r',
    s=30,
    alpha=0.7,
    edgecolors='black',
    linewidth=0.5
)
plt.colorbar(scatter, label='Habitat Suitability')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Habitat Suitability')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()



Fitting One-Class SVM...
Model fitting completed.
Model AUC Score: 0.973


NameError: name 'plt' is not defined